##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API：嵌入快速入門


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/gemini-api-cookbook/blob/zh-tw/quickstarts/Embeddings.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />執行於 Google Colab</a>
  </td>
</table>


Gemini API 會生成最先進的文字嵌入。嵌入式是一串浮點數，代表詞彙、句子或段落的意思。你可以在多個下遊應用程式中使用嵌入，例如文件搜尋。

這本筆記書提供快速程式碼範例，教你如何開始產生嵌入式。


In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 994.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 5.9 MB/s eta 0:00:00


In [2]:
import google.generativeai as genai

## 設定你的 API 鍵

若要執行以下單元格，必須將你的 API 鍵儲存在名為 `GOOGLE_API_KEY` 的 Colab 機密中。如果你還沒有 API 鍵，或者不確定如何建立 Colab 機密，請參閱 [驗證](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) 以取得範例。


In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## 嵌入內容

呼叫 `embed_content` 方法與 `models/text-embedding-004` 模型以產生文字嵌入。


In [4]:
text = "Hello world"
result = genai.embed_content(model="models/text-embedding-004", content=text)

# Print just a part of the embedding to keep the output manageable
print(str(result['embedding'])[:50], '... TRIMMED]')

[0.013168523, -0.008711934, -0.046782676, 0.000699 ... TRIMMED]


In [5]:
print(len(result['embedding'])) # The embeddings have 768 dimensions

768


## 一批次嵌入內容

你可以使用一個 API 呼叫來嵌入多個提示列表以提高效率。


In [6]:
result = genai.embed_content(
    model="models/text-embedding-004",
    content=[
      'What is the meaning of life?',
      'How much wood would a woodchuck chuck?',
      'How does the brain work?'])

for embedding in result['embedding']:
  print(str(embedding)[:50], '... TRIMMED]')

[-0.010632277, 0.019375855, 0.0209652, 0.000770642 ... TRIMMED]
[0.018467998, 0.0054281196, -0.017658804, 0.013859 ... TRIMMED]
[0.05808907, 0.020941721, -0.108728774, -0.0403925 ... TRIMMED]


## 截斷 embeddings

`text-embedding-004` 模型也支援較低維度的 embedding。指定 `output_dimensionality` 可以截斷輸出。


In [7]:
# Not truncated
result1 = genai.embed_content(
    model="models/text-embedding-004",
    content="Hello world")


# Truncated
result2 = genai.embed_content(
    model="models/text-embedding-004",
    content="Hello world",
    output_dimensionality=10)


(len(result1['embedding']), len(result2['embedding']))

(768, 10)

## 指定 `task_type`


讓我們看看 `embed_content` 方法所採取的所有參數。共有五個：

* `model`: 必填。必須是 `models/text-embedding-004` 或 `models/embedding-001`。
* `content`: 必填。你想要內嵌的內容。
*`task_type`: 可選。將會使用內嵌的任務類型。
* `title`: 可選。只有當你的任務類型為 `retrieval_document` (或 `document`) 時才應設定此參數。
* `output_dimensionality`: 可選。輸出內嵌的縮減維數。如果設定，輸出內嵌中的多餘值會從末端被截斷。這由 `models/text-embedding-004` 支援，但無法在 `models/embedding-001` 中指定。

`task_type` 是個可選參數，提供應用程式中使用內嵌方式的提示。

接受下列 task_type 參數：

* `unspecified`: 如果你未設定值，它會預設為 `retrieval_query`。
* `retrieval_query` (或 `query`)：給定文字是搜尋/擷取設定中的查詢。
* `retrieval_document` (或 `document`)：給定文字是正在搜尋的語料庫文件。選擇性地，同時使用文件的標題設定 `title` 參數。
* `semantic_similarity` (或 `similarity`)：給定文字將用於語意文字相似性 (STS)。
* `classification`: 分類給定文字。
* `clustering`: 將內嵌用於分群。
* `question_answering`: 將給定文字用於回答問題。
* `fact_verification`: 將給定文字用於驗證事實。


In [10]:
# Notice the API returns different embeddings depending on `task_type`
result1 = genai.embed_content(
    model="models/text-embedding-004",
    content="Hello world")

result2 = genai.embed_content(
    model="models/text-embedding-004",
    content="Hello world",
    task_type="document")

print(str(result1['embedding'])[:50], '... TRIMMED]')
print(str(result2['embedding'])[:50], '... TRIMMED]')

[0.013168523, -0.008711934, -0.046782676, 0.000699 ... TRIMMED]
[0.023399517, -0.00854715, -0.052534223, -0.012143 ... TRIMMED]


## 進一步學習

查看 Cookbook 中的這些範例，進一步了解你可以使用嵌入式執行的動作：

* [搜尋重新排名](https://github.com/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb)：使用 Gemini API 中的嵌入式從 Wikipedia 重新排名搜尋結果。

* [使用嵌入式進行異常檢測](https://github.com/google-gemini/cookbook/blob/main/examples/Anomaly_detection_with_embeddings.ipynb)：使用 Gemini API 中的嵌入式檢測出資料集中的潛在異常值。

* [訓練文字分類器](https://github.com/google-gemini/cookbook/blob/main/examples/Classify_text_with_embeddings.ipynb)：使用 Gemini API 中的嵌入式訓練一個模型，此模型可以根據主題分類不同類型的新聞小組文章。

* 嵌入式在向量資料庫中也有許多應用程式。查看這個 [使用 Chroma DB 的範例](https://github.com/google/generative-ai-docs/blob/main/examples/gemini/python/vectordb_with_chroma/vectordb_with_chroma.ipynb)。

你可以在 ai.google.dev 的 [嵌入式指南](https://ai.google.dev/docs/embeddings_guide) 進一步了解一般性的嵌入式

* 你可以在 [這裡](https://ai.google.dev/tutorials/python_quickstart#use_embeddings) 找到使用 Python SDK 的其他程式碼範例。

* 你也可以在 [embedContent](https://ai.google.dev/api/rest/v1/models/embedContent) 和 [batchEmbedContents](https://ai.google.dev/api/rest/v1/models/batchEmbedContents) 的 API 參考中找到更多詳細資訊。
